In [1]:
import pandahouse
import pandas as pd
import numpy as np
from scipy import stats
from tqdm import tqdm  # Прогресс-бар
import pandahouse
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import hashlib # Делать хеши
import swifter # Ускорялка метода apply
from scipy import stats # Статистика
from scipy.stats import norm, ttest_ind
%matplotlib inline
connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20250220'
}

Задача: улучшение чувствительности A/B-теста

Ранее я анализировала все данные, включая пользователей, на которых экспериментальный алгоритм не мог повлиять (например, с числом просмотров меньше 30). В этом задании пересматриваю подход: отбираю только тех пользователей, для которых алгоритм действительно мог сработать, и анализирую только их поведение.

Цель состояла в том, чтобы повысить чувствительность A/B-теста, исключив нерелевантные данные и уменьшив шум. Я смоделировала поведение пользователей (просмотры, CTR, лайки) и сравнила группы только среди тех, кто мог быть затронут алгоритмом.

В результате оценила текущую мощность теста и сделала выводы о том, как такой фильтр влияет на точность и чувствительность анализа.


In [2]:
# Распределение просмотров пользователей
q_views = """
select views, count() as users
from (select
    user_id,
    sum(action = 'view') as views
from {db}.feed_actions
where toDate(time) between '2025-01-24' AND '2025-01-30'
group by user_id
)
group by views
order by views
"""
views = pandahouse.read_clickhouse(q_views, connection=connection)

# Распределение поюзерного CTR за A/A-период
q_ctr = """
select 
   floor(ctr, 2) as ctr, count() as users
from (select toDate(time) as dt, 
    user_id,
    sum(action = 'like')/sum(action = 'view') as ctr
from {db}.feed_actions
where dt between '2025-01-24' AND '2025-01-30'
group by dt,user_id
)
group by ctr
"""
ctr = pandahouse.read_clickhouse(q_ctr, connection=connection)

# Нормирование вероятности для дискретных распределений
views['p'] = views['users'] / views['users'].sum() 
ctr['p'] = ctr['users'] / ctr['users'].sum() 

In [3]:
# Количество пользователей за весь период 24.01-06.02
q_count_user= """
SELECT count(DISTINCT user_id) as total_users
FROM {db}.feed_actions
WHERE toDate(time) BETWEEN '2025-01-24' AND '2025-02-06'

"""

count_users = pandahouse.read_clickhouse(q_count_user, connection=connection)
total_users = int(count_users.iloc[0, 0])

In [4]:
rng = np.random.default_rng()

n=total_users//2 # Размер каждой группы
num_simulations = 20000
result_list = []
    
for i in tqdm(range(num_simulations)):
    # Сэмплирование просмотров с учетом вероятностей
    group_A_views = rng.choice(views['views'], size=n, replace=True, p=views['p']).astype(np.int64)
    group_B_views = rng.choice(views['views'], size=n, replace=True, p=views['p']).astype(np.int64)
    
    # Применение эффекта алгоритма в группе B
    group_B_views = group_B_views + ((1 + rng.binomial(1, 0.5, size=n)) * rng.binomial(1, 0.9, size=n)) * (group_B_views >= 30)
    
    # Сэмплирование CTR
    group_A_ctr = rng.choice(ctr['ctr'], size=n, replace=True, p=ctr['p'])
    group_B_ctr = rng.choice(ctr['ctr'], size=n, replace=True, p=ctr['p'])
    
    # Генерация лайков на основе просмотров и CTR
    group_A_likes = rng.binomial(group_A_views, group_A_ctr)
    group_B_likes = rng.binomial(group_B_views, group_B_ctr)
    
    # Отбор пользователей с ≥30 просмотрами
    mask_A = group_A_views >= 30
    mask_B = group_B_views >= 30
    
    # Проведение t-теста для отфильтрованных пользователей
    result_list.append(stats.ttest_ind(group_A_likes[mask_A], group_B_likes[mask_B], equal_var=False).pvalue < 0.05)

# Расчет мощности теста
result = (sum(result_list) / num_simulations) * 100
print(f"Финальный результат: мощность теста {result:.2f} %")

100%|██████████| 20000/20000 [07:18<00:00, 45.59it/s]

Финальный результат: мощность теста 64.07 %


Мощность теста после фильтрации пользователей с количеством просмотров менее 30 составила 64.07%. Таким образом, исключение нерелевантных пользователей позволяет сделать тест более чувствительным, несмотря на уменьшение размера выборки. Этот подход помогает точнее оценить влияние алгоритма на целевую аудиторию.